In [1]:
from seleniumbase import Driver
from seleniumbase import page_actions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import requests
import time
import csv
import os
from pathlib import Path
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
import pypandoc
from pypandoc.pandoc_download import download_pandoc

main_path = "data/"

class Leetcode:
    def __init__(self):
        self.driver = Driver(uc=True)
        self.tag_list=[]
        self.companies_list=[]
        self.qlno =1
        self.question_dict_list={}
        self.qtno = 0
        self.cno = 0
        self.tno = 0 
    # this for login
    def Login(self,id,pw):
        url = "https://leetcode.com/accounts/login/"
        self.driver.get(url)
        id_field = self.driver.find_element(by=By.ID, value="id_login")
        id_field.send_keys(id)
        time.sleep(1)
        pw_field = self.driver.find_element(by=By.ID, value="id_password")
        pw_field.send_keys(pw)
        #no need check ctp
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.ID, "signin_btn"))).click()
        time.sleep(5)
     # this for get all tag and companies list
    def GetAllTagAndCompaniesList(self):
        url = "https://leetcode.com/problemset/"
        self.driver.get(url)
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, "(//button[@id='headlessui-popover-button-:r5:'])[1]"))).click()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[@class='text-blue dark:text-dark-blue m-1 flex cursor-pointer items-center px-1'][normalize-space()='Expand'])[1]"))).click()
        tag_elements_parent = self.driver.find_element(by = By.XPATH, value="(//div[@class='-m-1 mt-1 flex max-h-[400px] flex-wrap overflow-auto py-4'])[1]")
        tag_elements = tag_elements_parent.find_elements(by = By.TAG_NAME, value="span")
        with open(main_path + 'tag.txt', 'w') as f:
            for e in tag_elements:
                self.tag_list.append(e.text)
                f.write(e.text)
                f.write('\n')
        f.close()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[@class='text-sm leading-5 font-normal text-label-2 dark:text-dark-label-2'])[1]"))).click()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[contains(text(),'Expand')])[1]"))).click()
        companies_elements_parent = self.driver.find_element(by = By.XPATH, value="(//div[@class='-m-1 mt-1 flex max-h-[400px] flex-wrap overflow-auto py-4'])[2]")
        companies_elements = companies_elements_parent.find_elements(by = By.TAG_NAME, value="span")
        with open(main_path + 'companies.txt', 'w') as f:
            for e in companies_elements:
                self.companies_list.append(e.text)
                f.write(e.text)
                f.write('\n')
        f.close()
        time.sleep(2)
        self.GetCompaniesQuestion()
    # this for converting companies name to url text format
    def GetCompaniesWebText(self,text):
        if text == "Quip (Salesforce)":
            return "quip"

        if text == "Virtu Financial":
            return "virtu"
            
        text = text.lower()
        text = text.replace(' ', '-')
        text = text.replace('.', '')
        return text
    # this for converting title name to url text format
    def GetTitleToWebText(self,text):
        text = text.lower()
        text = text.replace('`', '')
        text = text.replace("(", '')
        text = text.replace(")", '')
        text = text.replace(":", '')
        text = text.replace(",", '-')
        text = text.replace("'", '')
        text = text.replace("%", '')
        text = text.replace("&", '')
        text = text.replace(' ', '-')
        text = re.sub(r'(-{2,})', '-', text)
        return text
    # this for scarp all leetcode question list and merge to 1 csv file 
    def GetQuestionListWithoutTag(self):
        path = main_path + "tmp"
            if not os.path.exists(path):
                os.makedirs(path)
        if self.qlno < 29:
            url ="https://leetcode.com/problemset/?sorting=W3sic29ydE9yZGVyIjoiREVTQ0VORElORyIsIm9yZGVyQnkiOiJGUkVRVUVOQ1kifV0%3D&page=" + str(self.qlno)
            self.driver.get(url)
            time.sleep(10)
            with open(path +"/"+ str(self.qlno) + ".csv", 'w', newline='',encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(["No", "Title","Link","Tags","Acceptance","Difficulty","Frequency"])
                cols = self.driver.find_elements(by = By.XPATH,value="(//div[@role='cell'])")
                no = str("")
                title = str("")
                link = str("")
                acceptance = str("")
                difficulty = str("")
                i = 1
                for c in cols:
                    column = i % 6
                    if column == 2:
                        text = c.text
                        no = text.split('. ')[0]
                        title = text.split('. ')[1]
                        link = "https://leetcode.com/problems/" + self.GetTitleToWebText(title) +"/description/"
                    elif column == 4 :
                        acceptance = c.text
                    elif column == 5:
                        difficulty = c.text
                    elif column == 0:
                        if no == "2764":
                            writer.writerow(["2764", "is-array-a-preorder-of-some-binary-tree" ,"https://leetcode.com/problems/is-array-a-preorder-of-some-binary-tree/description/","","65.9%","Medium",target_text])
                        else:
                            html_text = c.get_attribute("innerHTML")
                            target_text = html_text.rsplit("style")[1]
                            target_text = target_text.split(";")[0]
                            target_text = target_text.replace('"','')
                            target_text = target_text.replace('=','')
                            target_text = target_text.replace('width','')
                            target_text = target_text.replace(':','')
                            target_text = target_text.replace(' ','')
                            writer.writerow([no, title ,link,"",acceptance,difficulty,target_text])
                    i +=1
                file.close()
            print(self.qlno)
            self.qlno +=1
            self.GetQuestionList()
        else:
            df1 = pd.read_csv(path + "/1.csv", encoding='utf-8')
            directory = path
            filenames = [f for f in listdir(directory) if isfile(join(directory, f))]
            for f in filenames:
                if f == "Question List Without Tag.csv" or f == "Question List.csv" or f == "1.csv" or f == "Question Tag.csv":
                    continue
                words = f.rsplit(".")
                extension = words[len(words) - 1]
                if extension == "csv":
                    df2 = pd.read_csv(path + "/" + f, encoding='utf-8')
                    df1 = pd.concat([df1, df2], ignore_index=True)
          
            for index, row in df1.iterrows():
                self.question_dict_list[str(row['Tags'])] =""
            df1.to_csv(path + "/Question List Without Tag.csv", encoding = 'utf-8')
    # this for getting tag for the question 
    def GetTagForTheQuestion(self):
        file_path = main_path + "tmp/Question Tag.csv"
        if self.qtno < len(self.question_dict_list):
            tag = ""
            url = self.find_question_tag_queue[self.qtno]
            self.driver.get(url)
            try:
                WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class='flex gap-1'] div:nth-child(2)"))).click()
                tag_elements_parent = self.driver.find_element(by = By.CSS_SELECTOR, value=".mt-2.flex.flex-wrap.gap-1.pl-7")
                time.sleep(1)
                tag_elements = tag_elements_parent.find_elements(by = By.TAG_NAME, value="a")
                for e in tag_elements:
                    text = "[" + e.text + "]"
                    tag += text
                file_exists = os.path.exists(file_path)
                if file_exists == False:
                    with open(file_path, 'w', newline='',encoding='utf-8') as file:
                        writer = csv.writer(file)
                        writer.writerow(["Link","Tags"])
                        writer = csv.writer(file)
                        writer.writerow([url,tag])
                        file.close()
                else:
                    with open(file_path, 'a', newline='',encoding='utf-8') as file:
                        writer = csv.writer(file)
                        writer.writerow([url,tag])
                        file.close()
            except:
                print(url,"no tag")
                file_exists = os.path.exists(file_path)
                if file_exists == False:
                    with open(file_path, 'w', newline='',encoding='utf-8') as file:
                        writer = csv.writer(file)
                        writer.writerow(["Link","Tags"])
                        writer = csv.writer(file)
                        writer.writerow([url,""])
                        file.close()
                else:
                    with open(file_path, 'a', newline='',encoding='utf-8') as file:
                        writer = csv.writer(file)
                        writer.writerow([url,""])
                        file.close()
            time.sleep(1)
            print(self.qtno) 
            self.qtno += 1
            self.GetTagForTheQuestion(file_path)
            
    # this for merge the tag info with the question list without tag and output question list.csv
    def GetQuestionList(self):
        df = pd.read_csv(main_path +"tmp/Question Tag.csv", encoding='utf-8')
        for index, row in df.iterrows():
            link = str(row['Link'])
            if link not in self.find_question_tag_dict_list:
                self.find_question_tag_dict_list[link] = str(row['Tags'])
        with open(main_path + "/Question List.csv", 'w', newline='',encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["No", "Title","Link","Tags","Acceptance","Difficulty","Frequency"])
            df = pd.read_csv(main_path +"tmp/Question List Without Tag.csv", encoding='utf-8')
            for index, row in df.iterrows():
                link = str(row['Link'])
                tags = str(row['Tags'])
                if link in self.find_question_tag_dict_list:
                    tags = self.find_question_tag_dict_list[link]
                writer.writerow([str(row['No']), str(row['Title']), str(row['Link']),tags,str(row['Acceptance']),str(row['Difficulty']),str(row['Frequency'])])
            file.close()
    # this for getting top 100 question
    def GetTop100Question(self):
        df1 = pd.read_csv( main_path + "/Question List.csv", encoding='utf-8')
        df2 = df1.head(100).copy()
        df2.to_csv(main_path + "/Top 100 Question List.csv", encoding = 'utf-8')
    # this for counting the tag frequency appearance in the quesiton list    
    def GetQuestionTagCount(self):
        df = pd.read_csv(main_path + "/Question List.csv", encoding='utf-8')
        tags_header_list = []
        top_dict = {}
        for index, row in df.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
            for tag in tags_list:
                if tag not in tags_header_list:
                    tags_header_list.append(tag)
                    top_dict[tag]=0
                top_dict[tag] += 1
        with open(main_path + "/Question List Tag Count.csv", 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Tag", "Appearance"])
            for s in sorted(top_dict, key=top_dict.get, reverse=True):
                writer.writerow([s, top_dict[s]])
            file.close()
            
        df = pd.read_csv(main_path + "/Top 100 Question List.csv", encoding='utf-8')
        tags_header_list.clear()
        top_dict.clear()
        for index, row in df.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
            for tag in tags_list:
                if tag not in tags_header_list:
                    tags_header_list.append(tag)
                    top_dict[tag]=0
                top_dict[tag] += 1
        with open(main_path + "/Top 100 Question List Tag Count.csv", 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Tag", "Appearance"])
            for s in sorted(top_dict, key=top_dict.get, reverse=True):
                writer.writerow([s, top_dict[s]])
            file.close()
    # this for scraping companies question list    
    def GetCompaniesQuestion(self):
        dest_folder = main_path + 'company-leetcode-question-list'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        
        if self.cno < len(self.companies_list):
            company_name = self.companies_list[self.ccno]
            file_name = company_name + ".csv"
            file_path = os.path.join(dest_folder, file_name)
            url = "https://leetcode.com/company/" + self.GetCompaniesWebText(company_name) + "/"
            self.driver.get(url)
            WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//input[@type='checkbox'])[1]"))).click()
            WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//th[normalize-space()='Frequency'])[1]"))).click()
            WebDriverWait(self.driver, 1).until(EC.element_to_be_clickable((By.XPATH, "(//th[normalize-space()='Frequency'])[1]"))).click()
            table = self.driver.find_element(by = By.CSS_SELECTOR, value=".reactable-data")
            rows = table.find_elements(by = By.TAG_NAME, value="tr")
            with open(file_path, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["No", "Title","Link","Tags","Acceptance","Difficulty","Frequency"])

                error =False
                for r in rows:
                    cols = r.find_elements(by = By.TAG_NAME, value="td")
                    if len(cols) == 7:
                        no = cols[1].text
                        title = cols[2].text
                        url = "https://leetcode.com/problems/" + self.GetTitleToWebText(title) +"/description/"
                        tags_list = cols[3].find_elements(by = By.TAG_NAME, value="a")
                        tags = ""
                        for t in tags_list:
                            tags = tags + "[" + t.text + "]"
                        acceptance = cols[4].text
                        difficulty = cols[5].text
                        frequency = cols[6].get_attribute("value")
                        
                        writer.writerow([no, title ,url,tags,acceptance,difficulty,frequency])
                    else:
                        error = True
                file.close()
            if error == True:
                self.failed_list.append(company_name)
                print("Data mining failed.",company_name)
            else:
                print("Done data mining",company_name)
                
            time.sleep(2)
            self.cno += 1
            self.GetCompaniesQuestion()
        else:
            self.driver.quit()
     # this for counting the tag frequency appearance in the company question list    
    def GetCompaniesQuestionTagCount(self):
        directory = main_path + "company-leetcode-question-list"
        filenames = [f for f in listdir(directory) if isfile(join(directory, f))]
        for f in filenames:
            df = pd.read_csv(directory + "/" + f, encoding='utf-8')
            tags_header_list = []
            top_dict = {}
            for index, row in df.iterrows():
                tags = str(row['Tags'])
                tags = tags.replace('[', '')
                tags_list = tags.split(']')
                if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                    tags_list= tags_list[:-1]
                if len(tags_list) == 0 or tags_list[0] =="nan":
                    continue
                for tag in tags_list:
                    if tag not in tags_header_list:
                        tags_header_list.append(tag)
                        top_dict[tag]=0
                    top_dict[tag] += 1
            with open(main_path + "/company-leetcode-question-tag-count" + f, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Tag", "Appearance"])
                for s in sorted(top_dict, key=top_dict.get, reverse=True):
                    writer.writerow([s, top_dict[s]])
                file.close()
     # this for prompt learning resource for all leetcode tag 
    def QueryCopliot(self):
        if self.tno < len(self.tag_list):
            keyword = self.tag_list[self.tno]
            url = "https://www.bing.com/search?q=Bing%20AI&showconv=1&form=MG0AUO"
            query = "Can you describe Leetcode " keyword +" tag type in one sentence and give me 5 reference links which I can learn for free." 
        
            with open(main_path + "tmp/" + keyword + ".html", 'w', encoding='utf-8') as file:
                self.driver.get(url)
                time.sleep(5)
                dom1 = self.driver.find_element(by=By.CSS_SELECTOR, value=".cib-serp-main").shadow_root
                input_shadow_dom2 = dom1.find_element(by=By.CSS_SELECTOR, value="#cib-action-bar-main").shadow_root
                input_shadow_dom3 = input_shadow_dom2.find_element(by=By.CSS_SELECTOR, value="cib-text-input[serp-slot='none']").shadow_root
                text_field = input_shadow_dom3.find_element(by=By.CSS_SELECTOR, value="#searchbox")
                text_field.send_keys(query)
                time.sleep(1)
                input_shadow_dom2.find_element(by=By.CSS_SELECTOR, value="button[aria-label='Submit']").click()
                time.sleep(30)
                response_shadow_dom2 = dom1.find_element(by=By.CSS_SELECTOR, value="#cib-conversation-main").shadow_root
                response_shadow_dom3 = response_shadow_dom2.find_element(by=By.CSS_SELECTOR, value="cib-chat-turn[serp-slot='none']").shadow_root
                response_shadow_dom4 = response_shadow_dom3.find_element(by=By.CSS_SELECTOR, value=".response-message-group").shadow_root
                response_shadow_dom5 = response_shadow_dom4.find_element(by=By.CSS_SELECTOR, value="cib-message:nth-child(2)").shadow_root
                response = response_shadow_dom5.find_element(by=By.CSS_SELECTOR, value="div[class='content']")
                content = response.get_attribute("innerHTML")
                file.write(content)  
                file.close()
                print(keyword,"\n")
            file.close()
            self.start += 1
            self.QueryCopliot()
    
     # this for combine all leetcode learning resource and tag count into 1 html file
    def GenerateLeetCodeLearningResource(self):
        html_content = ""
        html_content +="<h1><u><b>"
        html_content += "Overall Leetcode Tag Type Appear in the Question Count"
        html_content +="</b></u></h1>\n"
        html_content +="<table>\n"
        html_content +="<tr>\n"
        html_content +="  <th>Tag</th>\n"
        html_content +="  <th>Count</th>\n"
        html_content +="</tr>\n"
        df = pd.read_csv(main_path + "/Question List Tag Count.csv")
        for index, row in df.iterrows():
            html_content += "<tr>\n" 
            tag_html ="  <td>" + str(row["Tag"]) + "</td>\n" 
            count_html ="  <td>" + str(row["Appearance"]) + "</td>\n" 
            html_content += tag_html
            html_content += count_html
            html_content += "</tr>\n"
        html_content +="</table>\n"

        html_content +="<h1><u><b>"
        html_content += "Top 100 Leetcode Question Tag Type Appear in the Question Count"
        html_content +="</b></u></h1>\n"
        html_content +="<table>\n"
        html_content +="<tr>\n"
        html_content +="  <th>Tag</th>\n"
        html_content +="  <th>Count</th>\n"
        html_content +="</tr>\n"
        df = pd.read_csv(main_path + "/Top 100 Question List Tag Count.csv")
        for index, row in df.iterrows():
            html_content += "<tr>\n" 
            tag_html ="  <td>" + str(row["Tag"]) + "</td>\n" 
            count_html ="  <td>" + str(row["Appearance"]) + "</td>\n" 
            html_content += tag_html
            html_content += count_html
            html_content += "</tr>\n"
        html_content +="</table>\n"

        directory = main_path + "tmp"
        filenames = [f for f in listdir(directory) if isfile(join(directory, f))]
        for f in filenames:
            words = f.rsplit(".")
            extension = words[len(words) - 1]
            if extension == "html":
            with open(directory + "/" + f, "r", encoding="utf-8") as file:
                title = i
                html_content +="<h1><u><b>"
                html_content += title
                html_content +="</b></u></h1>"
                html_content += file.read()
            file.close()
        with open(main_path +"/leetcode learning resource.html", 'w', encoding='utf-8') as file:
            file.write(html_content)  
            file.close()
        output = pypandoc.convert_text(html_content, 'docx', format='html', outputfile=main_path+'/leetcode learning resource.docx')
        if output == "":
            print("Document output sucessfully.")
        else:
            print("Document output failed")
                
        